In [1]:
# 환경 설정
from dotenv import load_dotenv
import os
import pandas as pd
from ai_agent_v2 import DataAnayticsAssistant

# OpenAI API 키 로드
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

PROCESSED_DATA_PATH = "../output/stage1/processed_data_info.xlsx"
mart_name = "cust_intg"
def load_processed_data_info():
    """사전에 분석된 데이터 정보 로드"""
    if not os.path.exists(PROCESSED_DATA_PATH):
        return None
    else:
        # 모든 시트 로드
        return pd.read_excel(PROCESSED_DATA_PATH, sheet_name=mart_name)


# ✅ Streamlit 실행 시 데이터 로드
processed_data_info = load_processed_data_info()

# 어시스턴트 초기화
assistant = DataAnayticsAssistant(openai_api_key, processed_data_info )

df = pd.read_pickle(f'../data/{mart_name}.pkl')

# 데이터프레임 설정
assistant.set_active_mart(df, mart_name)


processed_data_info        데이터프레임명         컬럼명   데이터 타입   count        mean         std  \
0    cust_intg        고객ID    int64  120000    43126.17    33176.95   
1    cust_intg          나이   object  120000         NaN         NaN   
2    cust_intg          성별   object  120000         NaN         NaN   
3    cust_intg       수익자여부    int64  120000        0.89        0.31   
4    cust_intg      CB신용평점  float64   14118      316.21      280.35   
..         ...         ...      ...     ...         ...         ...   
118  cust_intg    변액종신보유여부    int64  120000        0.06        0.24   
119  cust_intg  변액종신최대납입회차  float64  120000       12.36       12.87   
120  cust_intg   변액종신유지계약수  float64  120000        0.08        0.29   
121  cust_intg  변액종신기납입보험료  float64  120000  3124802.54  3134004.52   
122  cust_intg        기준년월    int64  120000   202407.50        1.71   

           min        25%         75%          max   결측 개수  결측치 비율  고유값 개수  \
0         1.00   14297.25    69490.25    100000.0

In [12]:
result = assistant.ask(f"""주요 변수를 선택하여 랜덤포레스트 기반 특징 중요도를 계산해 주세요.""")
print(result['messages'])
print(result['validated_code'])
print(result['insights'])
print(result['report_filename'])


🔄 새로운 요청 처리 시작: '주요 변수를 선택하여 랜덤포레스트 기반 특징 중요도를 계산해 주세요.'

👨‍💼 Supervisor 단계:
🏃🏿‍➡️ 다음 단계: Analytics

📊 [handle_analytics] 분석 요청 처리 시작
현재 데이터 정보:
 <class 'pandas.core.frame.DataFrame'>

🤖 코드 생성 단계:
✨ 생성된 코드:
```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# 데이터 전처리
df = df.dropna(subset=['수익자여부'])  # 타겟 변수가 NaN인 경우 제거
y = df['수익자여부']
X = df.drop(columns=['고객ID', '수익자여부', '기준년월'])

# 범주형 변수 인코딩
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))

# 결측값 처리
X = X.fillna(X.mean())

# 랜덤 포레스트 모델 초기화
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 학습
model.fit(X, y)

# 특징 중요도 추출
importances = model.feature_importances_

# 중요도 데이터프레임 생성
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importances = feature_importances.sort_values(by='import

,feature,importance
0,미납보험료합계,0.027759
1,변액종신기납입보험료,0.027639
2,보험계약대출잔액합계,0.026451
3,변액종신CMIP,0.026389
4,변액기납입보험료,0.025921



📂 처리 데이터 저장 단계

🔄 인사이트 도출 단계:


,feature,importance
0,미납보험료합계,0.027759
1,변액종신기납입보험료,0.027639
2,보험계약대출잔액합계,0.026451
3,변액종신CMIP,0.026389
4,변액기납입보험료,0.025921


🌀 생성된 인사이트:
1. 주요 발견사항:
   - 이번 분석에서 가장 중요한 변수로 꼽힌 것은 '미납보험료합계'로, 중요도 점수가 0.027759입니다. 이는 보험계약자의 미납 보험료 상태가 보험 관련 의사결정이나 위험 평가에 큰 영향을 미칠 수 있음을 시사합니다.
   - '변액종신기납입보험료'와 '보험계약대출잔액합계' 역시 높은 중요도를 가진 변수들로, 각각 0.027639와 0.026451의 점수를 기록했습니다. 이는 변수들이 보험 계약자의 경제적 상태나 보험 상품의 지속성에 크게 기여할 수 있다는 것을 암시합니다.
   - '변액종신CMIP' 및 '변액기납입보험료'도 상대적으로 중요한 변수로 나타났으며, 이는 변액 보험료와 관련된 변수들이 계약자의 보험 계약 및 유지에 영향을 미칠 수 있음을 보여줍니다.

2. 특이점:
   - 각 변수의 중요도 차이가 크지 않으며, 모두 비슷한 범위 내에서 나타났습니다. 이는 분석 전반에 걸쳐 여러 변수가 대부분의 모델 성능에 기여하고 있다는 것을 의미할 수 있습니다.
   - 특히 변액보험 관련 변수들이 중요도 순위 상위에 집중되어 있어, 변액보험 상품의 특성이나 계약자 행동이 매우 중요한 역할을 하고 있을 가능성이 있습니다.

3. 추천 사항:
   - 미납보험료와 관련된 리스크를 더욱 세밀하게 관리하기 위한 추가 분석이 필요할 수 있습니다. 이를 통해 해당 변수가 보험 손익에 미치는 영향을 더욱 명확히 이해할 수 있습니다.
   - 변액보험 관련 변수들이 중요하게 나타난 만큼, 변액보험 상품의 설계 및 관리에 대한 전략을 재검토할 필요가 있습니다. 특히 변액종신보험과 관련된 요인을 심층 분석하여 고객 맞춤형 상품 제공 및 유지 전략을 마련해야 합니다.
   - '보험계약대출잔액합계'의 중요성을 고려할 때, 계약자 대출과 관련된 리스크 평가 및 관리 방안을 강화하는 것도 추천합니다. 대출 상환 능력에 대한 정교한 평가 기준을 마련하면, 장기적인 손실을 줄일 수 있을 것입니다.
🌀 차트 필요 여부: 'yes'

🔄

In [19]:
print(f"#####   messages:\n  {result['messages'][0].content}")
print(f"#####   validated_code:\n  {result['validated_code']}")
print(f"#####   insights:\n  {result['insights']}")
print(f"#####   report_filename:\n  {result['report_filename']}")


#####messages:
  주요 변수를 선택하여 랜덤포레스트 기반 특징 중요도를 계산해 주세요.
#####validated_code:
  ```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# 데이터 전처리
df = df.dropna(subset=['수익자여부'])  # 타겟 변수가 NaN인 경우 제거
y = df['수익자여부']
X = df.drop(columns=['고객ID', '수익자여부', '기준년월'])

# 범주형 변수 인코딩
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))

# 결측값 처리
X = X.fillna(X.mean())

# 랜덤 포레스트 모델 초기화
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 모델 학습
model.fit(X, y)

# 특징 중요도 추출
importances = model.feature_importances_

# 중요도 데이터프레임 생성
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': importances})
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# 결과 저장
result_df = feature_importances.reset_index(drop=True)
```
#####insights:
  1. 주요 발견사항:
   - 이번 

In [11]:
print(result['report_filename'])

# 보고서

## 1. 요약

이 보고서는 다양한 고객 데이터 정보를 가지고, 해당 데이터를 심층 분석하기 위한 기초 인사이트를 제공합니다. 데이터를 검토한 결과, 고객의 보험 계약 유지 패턴, 상품 선호도, 채널 선호도 분석이 가능하며, 이러한 정보를 통해 효율적인 마케팅 전략과 고객 유지 활동을 지원할 수 있습니다. 다만, 데이터의 특성상 결측치가 많은 일부 컬럼에 대한 추가적인 검토가 필요하며, 데이터의 일관성 및 최신성을 보장하기 위한 조치가 필요합니다.

## 2. 분석 방법

데이터프레임에서 불필요한 컬럼을 제거하여 데이터의 핵심 요소를 검토했습니다. 주요 분석 포인트는 고객의 인구통계학적 정보와 보험 상품 관련 정보, 그리고 다양한 채널별 활동 및 계약 정보입니다. 결측치가 많은 컬럼의 경우, 대체할 수 있는 전략과 결측치의 원인을 파악하는 데 주안점을 두었습니다. 전체 데이터를 기반으로 고객의 행동 패턴과 선호도를 분석하여 향후 마케팅 전략과 데이터 활용 방법을 제시하였습니다.

## 3. 주요 발견사항

- **다양한 정보 포함:** 고객 데이터는 다양한 채널 활동과 계약 정보, 지급 이력 및 보험 상품 관련 정보를 포함해, 고객 행동 분석에 적합한 기반을 제공합니다.
- **결측치 존재:** CB신용평점, 직업분류명 등의 컬럼에 결측치가 많아 이를 보완하기 위한 추가 분석 또는 데이터 처리가 필요합니다.
- **데이터 특성:** 특정 데이터가 [OLD] 표시로 확인되어, 데이터의 최신 여부와 향후 업데이트 필요성을 검토해야 합니다.
- **분석 잠재력:** 채널별 활동 여부 및 계약 정보를 통해 고객의 선호도를 분석하여, 차별화된 마케팅 전략을 수립할 수 있는 가능성을 제공합니다.

## 4. 결론 및 제언

- **결측치 처리:** CB신용평점과 같은 주요 컬럼의 결측치를 대체하거나 그 원인을 분석하여 추가적인 가치를 도출할 필요가 있습니다.
- **데이터 업데이트:** [OLD]로 표시된 데이터가 포함된 부분의 갱신을 통해 데이터의 일관성

In [3]:

# 분석 요청 실행
result = assistant.ask(f"""
제공한 데이터프레임을 이용하여 아래 각 단계별 항목에 대해 차례대로 적용 및 그 결과를 정리하고, 
각 단계는 데이터 분석가가 EDA(Exploratory Data Analysis)를 수행할 때 더욱 체계적이고 심층적으로 접근할 수 있도록 해달라.

                       
1. Remove Unnecessary Columns and List Up
데이터셋에서 모든 행에 대해 동일한 값을 가지는 열(예: 모든 행이 1인 열)을 식별하고 제거하세요. 
추가적으로, 분석 목적과 무관하거나 중복된 정보를 제공하는 열도 제거 대상으로 고려하세요.
제거된 열 목록을 정리하고, 제거 이유를 간단히 설명하세요.
추가 고려사항:
열의 고유값(unique values)을 확인하여 모든 행이 동일한 값을 가지는지 검토하세요.
열의 의미와 분석 목적을 고려하여, 불필요한 열인지 판단하세요.
제거 전, 해당 열이 향후 모델링이나 분석에 사용될 가능성이 없는지 다시 한번 확인하세요.

2. Check for Missing Values
데이터셋의 각 열에 대해 결측값(missing values)이 있는지 확인하세요.
결측값이 발견된 경우, 해당 열의 데이터 분포와 결측값의 비율을 고려하여 적절한 처리 전략을 제안하세요.
단순히 0이나 평균값, 중앙값으로 대체하는 것은 지양하세요.
결측값 처리 방법: 삭제, 다중 대체법(Multiple Imputation), 예측 모델을 활용한 대체 등 상황에 맞는 방법을 선택하세요.
결측값 처리 후, 데이터의 무결성과 분석 결과에 미치는 영향을 검토하세요.
추가 고려사항:
결측값이 발생한 이유(MCAR, MAR, MNAR)를 고려하여 처리 방법을 결정하세요.
범주형 변수의 경우, "Unknown" 또는 "Missing"과 같은 새로운 카테고리를 추가하는 방법도 고려하세요.
결측값이 많은 열은 전체 데이터 품질에 영향을 미칠 수 있으므로, 열 자체를 제거하는 것도 고려하세요.

3. Data Type Consistency
데이터셋의 각 열이 올바른 데이터 타입을 가지고 있는지 확인하세요.
범주형 변수(categorical variables)는 object 또는 category 타입으로 변환하세요.
수치형 변수(numerical variables)는 int 또는 float 타입으로 변환하세요.
날짜/시간 데이터는 datetime 타입으로 변환하세요.
잘못된 데이터 타입으로 인해 발생할 수 있는 문제(예: 수치형 데이터가 문자열로 저장된 경우)를 해결하세요.
데이터 타입 변환 시, 데이터 손실이나 오류가 발생하지 않도록 주의하세요.
범주형 변수의 경우, 고유값의 수를 확인하여 적절한 인코딩 방법(예: One-Hot Encoding, Label Encoding)을 선택하세요.
데이터 타입 변환 후, 데이터의 무결성을 다시 한번 검토하세요.

4. Duplicate Rows
데이터셋에서 중복된 행(duplicate rows)이 있는지 확인하세요.
중복된 행이 발견된 경우, 해당 행을 제거하세요.
중복 행 제거 후, 데이터셋의 크기와 무결성을 검토하세요.
추가 고려사항:
중복 행을 제거하기 전, 중복이 발생한 이유를 분석하세요(예: 데이터 수집 오류, 병합 과정에서의 중복 등).
특정 열을 기준으로 중복을 판단할 경우, 해당 열의 중요성을 고려하세요.
중복 행 제거 후, 데이터셋의 균형(예: 클래스 불균형)이 깨지지 않았는지 확인하세요.

5. Outlier Detection
수치형 열에서 이상치(outliers)를 식별하세요.
이상치를 처리하기 전, 해당 열의 도메인 지식과 데이터 분포를 고려하여 이상치가 실제로 유효한 값인지 판단하세요(예: 특정 직군의 높은 연봉).
이상치 처리 방법: 제거, 변환(예: 로그 변환), 또는 별도의 범주로 처리하는 방법을 고려하세요.
이상치 처리 후, 데이터의 분포와 분석 결과에 미치는 영향을 검토하세요.
추가 고려사항:
이상치를 식별하기 위해 IQR(Interquartile Range), Z-score, 또는 시각화(예: 박스플롯)를 활용하세요.

이상치가 특정 패턴이나 의미를 가질 경우, 이를 별도로 분석하거나 모델링에 반영하세요.
이상치 처리 방법을 선택할 때, 데이터의 특성과 분석 목적을 고려하세요.
"""
)
# 결과 확인
print(result)

NameError: name 'assistant' is not defined

In [6]:
# 환경 설정
from dotenv import load_dotenv

# OpenAI API 키 로드
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')

# 어시스턴트 초기화
assistant = AIDataFrameAssistant(openai_api_key)

df = pd.read_pickle(f'../data/cust_intg.pkl')



🔨 그래프 구성 시작
✅ 노드 추가 완료
✅ 엣지 설정 완료
✅ 그래프 컴파일 완료



In [7]:
import pandas as pd
import numpy as np
import time
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

print("데이터 전처리 시작...")
start_time = time.time()

# 1. Remove Unnecessary Columns and List Up
print("\n1. 불필요한 컬럼 제거 시작...")
step_start = time.time()

cols_to_remove = []
unique_value_cols = df.columns[df.nunique() <= 1]
cols_to_remove.extend(unique_value_cols)
unnecessary_cols = ['기준년월']
cols_to_remove.extend(unnecessary_cols)
df = df.drop(columns=cols_to_remove)

print(f"제거된 컬럼: {cols_to_remove}")
print(f"소요 시간: {time.time() - step_start:.2f}초")

# 2. Check for Missing Values
print("\n2. 결측치 처리 시작...")
step_start = time.time()

missing_values_count = df.isnull().sum()
print(f"결측치 현황:\n{missing_values_count}")

numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print("수치형 변수 결측치 처리 중...")
imp = IterativeImputer(estimator=RandomForestRegressor(), random_state=0, max_iter=10, tol=1e-3)
df[numerical_cols] = imp.fit_transform(df[numerical_cols])

# 큐가 오래걸릴법한 애는 튜닝해서 줘.

print("범주형 변수 결측치 처리 중...")
for col in cat_cols:
    df[col].fillna('Unknown', inplace=True)

print(f"소요 시간: {time.time() - step_start:.2f}초")

# 3. Data Type Consistency
print("\n3. 데이터 타입 변환 시작...")
step_start = time.time()

df[cat_cols] = df[cat_cols].astype('category')
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"소요 시간: {time.time() - step_start:.2f}초")

# 4. Duplicate Rows
print("\n4. 중복 행 제거 시작...")
step_start = time.time()

initial_rows = len(df)
df.drop_duplicates(inplace=True)
removed_rows = initial_rows - len(df)

print(f"제거된 중복 행 수: {removed_rows}")
print(f"소요 시간: {time.time() - step_start:.2f}초")

# 5. Outlier Detection
print("\n5. 이상치 처리 시작...")
step_start = time.time()

initial_rows = len(df)
for col in numerical_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

removed_outliers = initial_rows - len(df)
print(f"제거된 이상치 행 수: {removed_outliers}")
print(f"소요 시간: {time.time() - step_start:.2f}초")

result_df = df.copy()

total_time = time.time() - start_time
print(f"\n전체 데이터 전처리 완료. 총 소요 시간: {total_time:.2f}초")

데이터 전처리 시작...

1. 불필요한 컬럼 제거 시작...
제거된 컬럼: ['GA채널Affluent고객여부', '자사설계사채널Affluent고객여부', '기준년월']
소요 시간: 0.25초

2. 결측치 처리 시작...
결측치 현황:
고객ID               0
나이                 0
성별                 0
수익자여부              0
CB신용평점        105882
               ...  
변액종신CMIP           0
변액종신보유여부           0
변액종신최대납입회차         0
변액종신유지계약수          0
변액종신기납입보험료         0
Length: 120, dtype: int64
수치형 변수 결측치 처리 중...


KeyboardInterrupt: 